In [1]:
# !pip install --extra-index-url=https://pypi.nvidia.com cudf-cu11==24.2.* dask-cudf-cu11==24.2.* cuml-cu11==24.2.*  cugraph-cu11==24.2.* cuspatial-cu11==24.2.* cuproj-cu11==24.2.* cuxfilter-cu11==24.2.* cucim-cu11==24.2.* pylibraft-cu11==24.2.* raft-dask-cu11==24.2.*

# Imports

In [1]:
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, auc
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import keras
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
import pandas as pd
from sklearn.model_selection import train_test_split
from time import time
from keras.utils import to_categorical
import numpy as np

# Loading Data

In [2]:
data = pd.read_csv("balanced_data.csv")
data.head()

,Unnamed: 0,case_id,bankacctype_710L,cardtype_51L,clientscnt_136L,credtype_322L,datefirstoffer_1144D,datelastinstal40dpd_247D,datelastunpaid_3546854D,disbursementtype_67L,...,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A
0,14670,0,1,2,5,3,1404,5175,5231,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0
1,637908,1602978,1,2,5,3,1404,5175,5231,3,...,0.0,0.0,15998.0,0.0,2.0,0.0,0.0,0.0,35865.344,0.0
2,266413,1602977,1,2,5,3,1404,5175,5231,3,...,0.0,1.0,50000.0,1.0,4.0,27662.8,27662.8,27662.8,60912.800,22775.6
3,842720,1602976,1,2,5,3,1404,5175,5231,3,...,0.0,0.0,40650.0,0.0,1.0,0.0,0.0,0.0,88454.600,0.0
4,421631,1602975,1,2,5,3,1404,5175,5231,3,...,0.0,0.0,23998.0,0.0,0.0,0.0,0.0,0.0,87624.980,0.0


In [3]:
data.drop("Unnamed: 0", axis = 1, inplace=True)
data.head()

,case_id,bankacctype_710L,cardtype_51L,clientscnt_136L,credtype_322L,datefirstoffer_1144D,datelastinstal40dpd_247D,datelastunpaid_3546854D,disbursementtype_67L,dtlastpmtallstes_4499206D,...,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A
0,0,1,2,5,3,1404,5175,5231,3,4464,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0
1,1602978,1,2,5,3,1404,5175,5231,3,4464,...,0.0,0.0,15998.0,0.0,2.0,0.0,0.0,0.0,35865.344,0.0
2,1602977,1,2,5,3,1404,5175,5231,3,4464,...,0.0,1.0,50000.0,1.0,4.0,27662.8,27662.8,27662.8,60912.800,22775.6
3,1602976,1,2,5,3,1404,5175,5231,3,4464,...,0.0,0.0,40650.0,0.0,1.0,0.0,0.0,0.0,88454.600,0.0
4,1602975,1,2,5,3,1404,5175,5231,3,4464,...,0.0,0.0,23998.0,0.0,0.0,0.0,0.0,0.0,87624.980,0.0


# Splitting the dataset

In [4]:
x = data.drop(['target'], axis = 1)
y = data['target']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

# SVM

In [6]:
modelSVC = LinearSVC()
start = time()
modelSVC.fit(x_train , y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_SVC = modelSVC.predict(x_test)
acc = accuracy_score(pred_SVC , y_test)
cm = confusion_matrix(pred_SVC , y_test)
cr = classification_report(pred_SVC , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

c:\ProgramData\anaconda3\envs\gpu\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


--------------------------------------------------------
Time taken: 58.28071141242981 seconds
--------------------------------------------------------
0.7245489019460766
--------------------------------------------------------
[[9170 3739]
 [2871 8217]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.76      0.71      0.74     12909
           1       0.69      0.74      0.71     11088

    accuracy                           0.72     23997
   macro avg       0.72      0.73      0.72     23997
weighted avg       0.73      0.72      0.72     23997

--------------------------------------------------------


c:\ProgramData\anaconda3\envs\gpu\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


# LGBM

In [7]:
modelLGBM = LGBMClassifier()
start = time()
modelLGBM.fit(x_train , y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_LGBM = modelLGBM.predict(x_test)
acc = accuracy_score(pred_LGBM , y_test)
cm = confusion_matrix(pred_LGBM , y_test)
cr = classification_report(pred_LGBM , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

[LightGBM] [Info] Number of positive: 36038, number of negative: 35953
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.048845 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 27895
[LightGBM] [Info] Number of data points in the train set: 71991, number of used features: 196
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500590 -> initscore=0.002361
[LightGBM] [Info] Start training from score 0.002361
--------------------------------------------------------
Time taken: 2.034799337387085 seconds
--------------------------------------------------------
0.9779555777805559
--------------------------------------------------------
[[12003   491]
 [   38 11465]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      0.96      0.98     12494
           1       0.96      1.00      0.98     11503

    accuracy              

# RF

In [8]:
model_RF = RandomForestClassifier(n_jobs=-1)
start = time()
model_RF.fit(x_train, y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_RF = model_RF.predict(x_test)
acc = accuracy_score(pred_RF , y_test)
cm = confusion_matrix(pred_RF , y_test)
cr = classification_report(pred_RF , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 6.78650689125061 seconds
--------------------------------------------------------
0.9729966245780722
--------------------------------------------------------
[[11972   579]
 [   69 11377]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.99      0.95      0.97     12551
           1       0.95      0.99      0.97     11446

    accuracy                           0.97     23997
   macro avg       0.97      0.97      0.97     23997
weighted avg       0.97      0.97      0.97     23997

--------------------------------------------------------


# ANN

In [47]:
# x_train2, y_train2, x_test2, y_test2 = x_train[:], y_train[:], x_test[:], y_test[:]
# y_train2 = tf.one_hot(y_train2, depth = 2)
# y_test2 = tf.one_hot(y_test2, depth = 2)

In [48]:
# x_train2.shape

In [10]:
model = keras.Sequential([
    keras.layers.Dense(1024, activation='relu', input_shape = (304,)),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(), loss=tf.keras.losses.BinaryCrossentropy(), metrics=keras.metrics.CategoricalAccuracy())

start = time()
history = model.fit(x_train, y_train, epochs=5, batch_size=32, use_multiprocessing=True, validation_split=0.5)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_ANN = model.predict(x_test)

Epoch 1/5
1125/1125 [==============================] - 11s 9ms/step - loss: 306153.8438 - categorical_accuracy: 1.0000 - val_loss: 30.7284 - val_categorical_accuracy: 1.0000
Epoch 2/5
1125/1125 [==============================] - 9s 8ms/step - loss: 7.9885 - categorical_accuracy: 1.0000 - val_loss: 9.4625 - val_categorical_accuracy: 1.0000
Epoch 3/5
1125/1125 [==============================] - 9s 8ms/step - loss: 1.2484 - categorical_accuracy: 1.0000 - val_loss: 8.6572 - val_categorical_accuracy: 1.0000
Epoch 4/5
1125/1125 [==============================] - 9s 8ms/step - loss: 1.2606 - categorical_accuracy: 1.0000 - val_loss: 2.8445 - val_categorical_accuracy: 1.0000
Epoch 5/5
1125/1125 [==============================] - 9s 8ms/step - loss: 15.0407 - categorical_accuracy: 1.0000 - val_loss: 1.0651 - val_categorical_accuracy: 1.0000
--------------------------------------------------------
Time taken: 46.47915720939636 seconds
--------------------------------------------------------
750/7

In [11]:
pred = []
for i in pred_ANN:
    pred.append(np.round(i[0]))

acc = accuracy_score(pred , y_test)
cm = confusion_matrix(pred , y_test)
cr = classification_report(pred , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

0.5429428678584823
--------------------------------------------------------
[[12038 10965]
 [    3   991]]
--------------------------------------------------------
              precision    recall  f1-score   support

         0.0       1.00      0.52      0.69     23003
         1.0       0.08      1.00      0.15       994

    accuracy                           0.54     23997
   macro avg       0.54      0.76      0.42     23997
weighted avg       0.96      0.54      0.66     23997

--------------------------------------------------------


# LR

In [11]:
model_lr = LogisticRegression(n_jobs = -1, max_iter=1000)
start = time()
model_lr.fit(x_train, y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_lr = model_lr.predict(x_test)
acc = accuracy_score(pred_lr , y_test)
cm = confusion_matrix(pred_lr , y_test)
cr = classification_report(pred_lr , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 33.416215658187866 seconds
--------------------------------------------------------
0.8028503562945368
--------------------------------------------------------
[[10206  2896]
 [ 1835  9060]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.85      0.78      0.81     13102
           1       0.76      0.83      0.79     10895

    accuracy                           0.80     23997
   macro avg       0.80      0.81      0.80     23997
weighted avg       0.81      0.80      0.80     23997

--------------------------------------------------------


# DT

In [12]:
model_DT = DecisionTreeClassifier()
start = time()
model_DT.fit(x_train, y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_DT = model_DT.predict(x_test)
acc = accuracy_score(pred_DT , y_test)
cm = confusion_matrix(pred_DT , y_test)
cr = classification_report(pred_DT , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 6.566051006317139 seconds
--------------------------------------------------------
0.9598699837479685
--------------------------------------------------------
[[11492   414]
 [  549 11542]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.95      0.97      0.96     11906
           1       0.97      0.95      0.96     12091

    accuracy                           0.96     23997
   macro avg       0.96      0.96      0.96     23997
weighted avg       0.96      0.96      0.96     23997

--------------------------------------------------------


# KNN

In [13]:
model_KN = KNeighborsClassifier()
start = time()
model_KN.fit(x,y)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_KN = model_KN.predict(x_test)
acc = accuracy_score(pred_KN , y_test)
cm = confusion_matrix(pred_KN , y_test)
cr = classification_report(pred_KN , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 0.3034937381744385 seconds
--------------------------------------------------------
0.9389923740467558
--------------------------------------------------------
[[11918  1341]
 [  123 10615]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.99      0.90      0.94     13259
           1       0.89      0.99      0.94     10738

    accuracy                           0.94     23997
   macro avg       0.94      0.94      0.94     23997
weighted avg       0.94      0.94      0.94     23997

--------------------------------------------------------


# NB

In [14]:
model_NB = KNeighborsClassifier()
start = time()
model_NB.fit(x,y)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_NB = model_NB.predict(x_test)
acc = accuracy_score(pred_NB , y_test)
cm = confusion_matrix(pred_NB , y_test)
cr = classification_report(pred_NB , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 0.26377224922180176 seconds
--------------------------------------------------------
0.9389923740467558
--------------------------------------------------------
[[11918  1341]
 [  123 10615]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.99      0.90      0.94     13259
           1       0.89      0.99      0.94     10738

    accuracy                           0.94     23997
   macro avg       0.94      0.94      0.94     23997
weighted avg       0.94      0.94      0.94     23997

--------------------------------------------------------


# MLP

In [15]:
model_NN = MLPClassifier()
start = time()
model_NN.fit(x,y)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_NN = model_NN.predict(x_test)
acc = accuracy_score(pred_NN , y_test)
cm = confusion_matrix(pred_NN , y_test)
cr = classification_report(pred_NN , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 100.15578079223633 seconds
--------------------------------------------------------
0.9243655456932116
--------------------------------------------------------
[[11364  1138]
 [  677 10818]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.94      0.91      0.93     12502
           1       0.90      0.94      0.92     11495

    accuracy                           0.92     23997
   macro avg       0.92      0.93      0.92     23997
weighted avg       0.93      0.92      0.92     23997

--------------------------------------------------------


# XGB

In [16]:
model_XG = xgb.XGBClassifier()
start = time()
model_XG.fit(x,y)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_XG = model_XG.predict(x_test)
acc = accuracy_score(pred_XG , y_test)
cm = confusion_matrix(pred_XG , y_test)
cr = classification_report(pred_XG , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 3.649543523788452 seconds
--------------------------------------------------------
0.9882485310663833
--------------------------------------------------------
[[12035   276]
 [    6 11680]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     12311
           1       0.98      1.00      0.99     11686

    accuracy                           0.99     23997
   macro avg       0.99      0.99      0.99     23997
weighted avg       0.99      0.99      0.99     23997

--------------------------------------------------------


# AdaBoost

In [17]:
model_ada = AdaBoostClassifier()
start = time()
model_ada.fit(x,y)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_ada = model_ada.predict(x_test)
acc = accuracy_score(pred_ada , y_test)
cm = confusion_matrix(pred_ada , y_test)
cr = classification_report(pred_ada , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

c:\ProgramData\anaconda3\envs\gpu\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


--------------------------------------------------------
Time taken: 36.055386781692505 seconds
--------------------------------------------------------
0.979997499687461
--------------------------------------------------------
[[12016   455]
 [   25 11501]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      0.96      0.98     12471
           1       0.96      1.00      0.98     11526

    accuracy                           0.98     23997
   macro avg       0.98      0.98      0.98     23997
weighted avg       0.98      0.98      0.98     23997

--------------------------------------------------------
